# Chapter 3: Making Mistakes, Faking Mistakes

At its core, this program allows the user to import images, modify them in various ways, see their progress, and save the result to a new image file. But how did I do that?

## Cellular Automata

Cellular automata can be seen as a kind of game, with pixels as players. Each pixel can be in one of two states: either on or off. Every round of the game, the pixels check their state, and the states of their neighbors. They look that information up in a specific table (this table is the "rule" of that particular game), and turn on or off accordingly.

There are one- and two-dimensional versions of this game. A famous two-dimensional rule is known as Conway's Game of Life. Despite it's simplicity, it is possible, though very difficult, to build complex structures within the game. "Although each cell has an extremely limited view of the system (just its immediate neighbours), localised information is propagated at each time step, enabling more global characteristics of the overall CA system" [(Rosin, "Image Processing", 790)](https://doi.org/10.1016/j.cviu.2010.02.005). For instance, in 2018, a group successfully built a working game of Tetris within Conway's Game of Life itself. [(Z. *et al.*)](https://codegolf.stackexchange.com/questions/11880/build-a-working-game-of-tetris-in-conways-game-of-life)

The one-dimensional version of the game is usually represented on a two-dimensional grid, with each row corresponding to one generation, with earlier generations at the top. As an example, here is the rulebook for a particular rule, called Rule 154, that I think looks particularly nice.

<img src="img/rule154.jpg" alt="The formats go through PIL, now." style="width: 500px;"/>

The example below shows Rule 154 in action. The dark brown pixels correspond to the black "on" pixels in the rulebook above, and the light blue pixels correspond to the white "off" pixels. If you look at any three consecutive pixels, you should be able to find a line in the rulebook above that corresponds to those pixels, and  the pixel below them will be the color indicated by the rulebook.

<img src="img/rule154detail.png" alt="Close up illustrating rule 154 and emerging patterns." style="width: 700px;"/>

Since this is the technical chapter, I suppose I need to give a bit of background into how I created these images. Essentially, I layered each generation over a new line of pixels in the image. When a pixel was "on," I inverted the colors of the image— so that white would become black, magenta would become green, and so forth. You can see that in the example above: the inversion of light blue/gray (the initial color) is dark orange/brown. However, as with any game, it isn't enough to have rules for how to make a move: you also need rules for how to set up the game to get ready to play. I decided to start with all pixels in the off state (where the color is not modified), but if a pixel is light enough, it flips to the on state (where the color will be inverted), at which point it will influence future pixels according to the rulebook.

Cellular automata have fascinated me for a long time, because of the complexity that's able to emerge from such simple rules. True, the cellular automata algorithms that I implemented in Glitcher don't resemble any sort of computer glitch I've ever heard of. That said, these algorithms embody the spirit of glitch art. Glitch art is about exploration, and about producing unexpected results from computer algorithms. From the simple rules of cellular automata, I always discover new and surpirisingly complex things. The collaborative nature of cellular automata— with each "player" following simple rules, but producing something larger and unexpected— also speaks to my philosophies about the human-computer interactions in glitch art.

In the full image below, we can see how Rule 154 affects this image of a seagull on a rock. (Note that I rotated this image 180 degrees, because I liked how that looked. As a result, the first round of the game is at the bottom, and the rounds move up instead of down.)

<img src="img/automata_seagull.png" alt="Full image, glitched with rule 154." style="width: 100%;"/>

## Data bending

Data bending is an integral part of glitch art. Peña et al. define data bending as "using software with the intention of disrupting a file at the raw data level." [(90)](https://go.gale.com/ps/anonymous?id=GALE%7CA497861449) For instance, one can open images in a text editor, manipulate them as text, and then open them as an image once again. Data bending was a really fun part of this project, and it produced some interesting results. Because of the complexity of image encodings, it was also one of the most unpredictable parts of this project.

I found it to be very easy to change the data in such a way that programs would be completely unable to display them. I found that some images would not display in the Preview app (the default image viewer on Apple computers), but would display when opened in Google Chrome. This was especially the case with BMP image files, since that format is primarily supported on Microsoft Windows computers. Some changes rendered files unreadable in any program.

I really wanted my Glitcher program to allow users to play with image files, with as much freedom as they could play with the other forms of glitches. This part of the program is still buggy, and I find it generally easier to data bend images manually, by opening the image as text, or in a hexidecimal reader. (I used Vim for reading and editing the hexidecimal, which I did when I wanted to change parts of the image header, which required more precision than I could easily get from text editors. The command `:%!xxd` will convert the image to hexidecimal. Then you can make changes, and save them to another file with the command `:%!xxd -r >outfile`. This makes it really easy to try things out, and get quick results.)

<table>
    <tr>
        <td><img src="img/test1.jpg" alt="simple test image"/>Test image</td>
        <td><img src="img/test1whby1.png" alt="lots of diagonal striping, weird colors"/>Increasing height by 1 and decreasing width by one in the header of a bmp image.</td>
        <td><img src="img/test1whdouble.png" alt="A very tall image, with lots of bands"/>Doubling the height and halving the width in the header of a bmp image.</td>
        <td><img src="img/test1bmpshuffle.png"/>The result of "shuffling" the bmp file by repeatedly cutting parts of the file as text, and pasting them elsewhere within the file.</td>
    </tr>
    <tr>
        <td><img src="img/test1shuffle.jpg"/>The result of "shuffling" a jpeg by repeatedly copying parts of the file as text, and pasting them elsewhere within the file.</td>
        <td><img src="img/test1jpegbyteflip.jpg"/>The result of randomly changing several characters to other characters in a jpeg image when opened as text.</td>
        <td><img src="img/test1inserttext.jpeg"/>The result of inserting a word into two random locations in a jpeg image, when opened as text.</td>
    </tr>
</table>

I found manipulating JPEG images to be much more unpredictable, and thus more exciting, than manipulating BMP images. JPEG images are much more compressed than BMP images: an image saved as a JPEG can be 20 times smaller than the same image stored as a BMP [(Miano, 35)](http://index-of.co.uk/Information-Theory/Compressed%20Image%20File%20Formats%20JPEG,%20PNG,%20GIF,%20XBM,%20BMP%20-%20John%20Miano.pdf). JPEG images are created by breaking the image into 8x8 pixel blocks, and processing these blocks individually, using a sum of cosine functions to approximate the block (Ibid. 44). This is why data bending JPEG images produces glitches as a grid of brightly colored blocks. Because of the complicated compression algorithms, I liked JPEG data bending more than BMP data bending: I couldn't exactly predict, or even explain, the results, so the process involved more experimentation.

## Dithering

What would you do if you had to display an image, but the pixels only had two options— on and off— instead of the normal gradation of brightness? The simplest answer would be just to designate some cutoff point— say, halfway between black and white, and have all pixels that are lighter than that cutoff be on, and all pixels that are darker than that cutoff be off.

<table>
<tr>
    <td><img src="img/eye.png" alt="test image of an eye" style="width:400px;"/>Test image<td>
    <td><img src="img/eye_cutoff.png" alt="test image of an eye" style="width:400px;"/>Test image with cutoff applied<td>
</tr>
</table>

This is a cool effect, and there are certainly situations where one might want it. But we lose a lot of detail, and almost all of the image is completely washed out.

The solution is a pointilist approach known as *dithering*. There are a number of clever dithering algorithms, but I stuck with one, called Bayer dithering, and explored what can be done within it. Like cellular automata, this algorithm is very simple: it is "a point process where computation uses only the current pixel in the input image to generate the corresponding pixel in the output image; no surrounding pixels are used" [(Ulichney, 1)](https://www.hpl.hp.com/techreports/Compaq-DEC/CRL-98-2.pdf). Once again, we find that complex and unexpected patterns emerge from these simple rules. For this algorithm, we create a matrix of numbers, and tile that matrix across the image. When the value of a pixel is greater than the value of the matrix element tiled to that pixel, we light it up, otherwise, it stays dark.

Let's see how this looks with a variety of matrices. First, here are some matrices that are known to give good results when used for Bayer dithering:

$M_0 = \begin{bmatrix}
0 & 2\\
3 & 1
\end{bmatrix}$

$M_1 = \begin{bmatrix}
{4 * M_0 + (M_0)_{1,1}} & {4 * M_0 + (M_0)_{1,2}}\\
{4 * M_0 + (M_0)_{2,1}} & {4 * M_0 + (M_0)_{2,2}}
\end{bmatrix} =
\begin{bmatrix}
{4 * M_0 + 0} & {4 * M_0 + 2}\\
{4 * M_0 + 3} & {4 * M_0 + 1}
\end{bmatrix} =$

$\begin{bmatrix}
0 & 8 & 2 & 10\\
12 & 4 & 14 & 6\\
3 & 11 & 1 & 9\\
15 & 7 & 13 & 5
\end{bmatrix}$

$M_2 = 
\begin{bmatrix}
{4 * M_1 + 0} & {4 * M_1 + 2}\\
{4 * M_1 + 3} & {4 * M_1 + 1}
\end{bmatrix} =
\begin{bmatrix}
0 & 32 & 8 & 40 & 2 & 34 & 10 & 42 \\
48 & 16 & 56 & 24 & 50 & 18 & 58 & 26  \\
12 & 44 & 4 & 36 & 14 & 46 & 6 & 38  \\
60 & 28 & 52 & 20 & 62 & 30 & 54 & 22  \\
3 & 35 & 11 & 43 & 1 & 33 & 9 & 41  \\
51 & 19 & 59 & 27 & 49 & 17 & 57 & 25  \\
15 & 47 & 7 & 39 & 13 & 45 & 5 & 37  \\
63 & 31 & 55 & 23 & 61 & 29 & 53 & 21  \\
\end{bmatrix}$

For higher values of $n$, I define $M_n$ in the same way, recursively applying the layout of $M_0$. (With all of these matrices, we also divide all numbers by the size of the matrix, so that the maximum value of the matrix corresponds to the maximum value that image pixels can take on.)

Notice that in all of these matricies, numbers are spread out as evenly as possible throughout the matrix. Each number is surrounded by numbers that are pretty far away from it, even when we tile the matrix. As a result, when I apply these matricies to images, there image has no lighter or darked clumps, where the higher or lower numbers are clustered on the matrix. The dithering algorithm does not draw attention to itself.

Of course, I was making glitch art, so algorithms that draw attention to themselves are exactly what I wanted. When I used other matricies where the low and high values are grouped together in some way, I got interesting results, as seen below.

<table>
    <tr>
    <td><img src="img/eye_M0.png" alt="Very pixellated dither" style="width: 350px;"/>
        $M_0$</td>
        <td><img src="img/eye_M1.png" alt="Getting more detailed..." style="width: 350px;"/>
        $M_1$</td>
        <td><img src="img/eye_M2.png" alt="and more natural..." style="width: 350px;"/>
        $M_2$</td>
        <td><img src="img/eye_M3.png" alt="pretty similar to m_2" style="width: 350px;"/>
        $M_3$</td>
    </tr>
    <tr>
        <td><img src="img/eye_increasing_square.png" alt="Big blotchy rows of color" style="width: 350px;"/>
        Square matrix, increasing in "reading order"</td>
        <td><img src="img/eye_increasing_row.png" alt="A lot like above, but verticle blotches" style="width: 350px;"/>
        One dimensional matrix, increasing along the row</td>
        <td><img src="img/eye_increasing_diagonal.png" alt="Lots of triangles!" style="width: 350px;"/>
        Square matrix, increasing along the diagonal.</td>
        <td><img src="img/eye_m2_newinital.png" alt="This is a cool one. We kind of have lots of thin vertical lines making up the image." style="width: 350px;"/>
        $M_2$, but if $M_0=\begin{bmatrix}0 & 1 & 2 & 3\\\end{bmatrix}$ and the recursive rule is the same.</td>
    </tr>
</table>

## Audio Methods

At a base level, all computer files, including images, video, text, and computer code, are just binary data in the computer's memory. The representation of that data depends wildly on the way the user tells the device to display that data. By treating one form of data as another, I created some interesting glitches. For instance, in the previous section, I glitched images by editing them as text.

One artist, writing under the pseudonym Gankra, describes a philosophy of "Right Value, Wrong Place" for creating glitches. "Rather than just inserting random changes or disabling some important check, take some of the real and correct values being fed into the code you're corrupting, and use them in a completely wrong place" [(Gankra)](https://gankra.github.io/glitch/). Using actual data for an incorrect purpose juxtaposes the structure of the data with the structure of the code, often revealing more about both of these structures. We can use this philosophy by processing one form of data as if it were a completely different form of data.

As I considered file formats to edit my images as, audio seemed like a promising choice. Digital audio editors, such as Audacity, FL Studio, or Garage Band, offer a wide range of audio effects, which could have interesting potential when applied to images. What would a reverb, or a low-pass filter, or even an auto-tune effect look like when applied to an image?

There are multiple ways I could have done this. The simplest would be to treat the image file as raw audio, and open it in an audio editor. The audio editor Audacity supports this. Then, I could apply any effects, export that as an audio file, import it back into Glitcher, and save it as an image. The result is... unimpressive. Here's what that method gave me on a JPEG file.

<img src="img/wav_from_rawjpg.png" alt="Jpeg image opened as a wav file" style="width: 500px;"/>

Not very recognizeable, now is it? JPEG images are generally very compressed, so the file has almost no resemblance to the image.

But what about other formats? BMP images are much more promising, since they have virtually no compression by default. Applying the same method to the test image saved to the BMP format gave me the following.

<img src="img/rawaudiobmp.png" alt="Jpeg image opened as a wav file" style="width: 500px;"/>

Very recognizeable! Unfortunately, this is still somewhat uninteresting. The colors are changed, and the image is flipped upside down, but not much is added. We have a line of random-looking pixels at the top of the image, corresponding to the header of the BMP file. While this was an interesting intellectual exercise in visualizing image formats, it lacks the aesthetics of glitch art.

To acchive these aesthetics, I had to be deliberate with how I converted between images and audio. Both of these formats are digital: they're made of discrete sampled values, represented as a number somewhere between the minimum and maximum values. 

<img src="img/two_to_one.jpg" alt="Two dimmensional array pointed to a one dimensional array" style="width: 500px;"/>

However, the dimensions of the formats aren't the same. Audio is mostly one-dimensional, exisiting along the dimension of time, while images are two-dimensional, with width and height. Turning an image into audio required flattening all those pixels into a sigle row. I would need to be able to reverse the steps, to convert the audio back into images. I decided to go with the simplest option, taking the rows of the image, putting them end, and saving that as an audio file.

<img src="img/whichrows.jpg" alt="Lots of ways to snake through a two dimensional array" style="width: 500px;"/>

There was still one more issue to consider. (There always is!) Each pixel of an image has three numbers, for the red, green, and blue channels. I decided to handle this in three different ways, and let the user decide which they wanted to use. In the first option, we go through the image once, storing the red, green, and blue values from each pixel together. For the second option, we go through the image three times, taking red on the first pass, then green, and finally blue. The third option is like the second, except each of those channels is actually stored in a separate audio file. 

To get a sense of what different audio effects can do, I applied several different audio effects to the same test image. (These use the second algorithm, where the red pixels go first, then the green pixels, and then the blue pixels.)

<table>
    <tr>
    <td><img src="img/hipass.png" alt="High Pass Filter on test image"/>
        High-pass filter</td>
        <td><img src="img/lopass.png" alt="Low Pass Filter on test image"/>
        Low-pass filter</td>
        <td><img src="img/delay.png" alt="Delay Effect on test image"/>
        Delay (echo) effect</td>
    </tr>
    <tr>
        <td><img src="img/downsample.png" alt="Downsample effect on test image"/>
        Downsample effect</td>
        <td><img src="img/autotune.png" alt="Autotune effect on test image"/>
        Autotune effect</td>
    </tr>
</table>
    

## Image Storage

In addition to creating the effects, I had to figure out how the effects would connect from one another, and how the project would be structured. One of the first questions I considered was how to store the images within the Glitcher object. There was no singular format that would work for everything I wanted to be able to do to the images. 
- For some methods, such as shuffling the bytes of the image file, the image needed to be encoded in a format such as JPEG. 
- For other methods, such as changing the contrast, I wanted to take advantage of the Python Image Library (PIL), which required the image representation to be stored as a PIL Image object. 
- For the bulk of my methods, I wanted to be able to interact with the pixels of the image as plainly as possible, which meant storing it as a three dimensional Numpy array: two dimensions for the width and height, and the third dimension for the RGB values of each pixel. Since I was writing this in Python, it was important to use a package such as Numpy, since manipulating arrays in Numpy is significantly faster than manipulating them as Python lists.

To handle and the conversion between these storage methods, I created a new class, called ImageStorage. When the Glitcher object needed the image to be in a specific format, it would call the ImageStorage method to fetch the image in that format. If the image was already in that format, the ImageStorage object would just return that image. Otherwise, it would convert it to the needed format.

Between Numpy arrays, PIL Images, and bytes for JPEG, PNG, and BMP image encodings, I had 5 formats, and I needed to be able to convert from any one of these formats to any other one. If I needed a separate function to convert from each method to each other method, I would have an unwieldy amount of code. Additionally, if I added a new format— say, say, GIF images— I didn't want to have to write separate methods to convert from that format to every other format.

<img src="img/conversion_problem.jpg" alt="All formats, with double arrows connecting each to each other. Very cluttered." style="width: 500px;"/>


To solve this problem, I used PIL Image objects as a middle ground. There were already built in methods for converting between PIL Images and Numpy arrays. PIL is built to be able to read and write image files, so I could set it up to read and write to byte objects instead. Now, when converting between two representations, we just convert to a PIL image, and then to the other format.

<img src="img/conversion_solution.jpg" alt="The formats go through PIL, now." style="width: 500px;"/>

## Checkpoints

I wanted playing with Glitcher to feel freeing. Glitches are generally associated with things breaking, and so can be tense experiences. With this project, however, I wanted to create code that allowed people to experiment without  anxiety.

To make that happen, it was important to have a way to undo actions. I implemented undoing by storing a list of copied Glitcher objects— essentially just shells around the corresponding ImageStorage representation. To set a checkpoint, the code just makes a new copy and pushes it on the end of the list. To undo to the most recent checkpoint, it pops the last element off the list, and sets that objects image as the current image.

To make it easier to go revert to an earlier chackpoint, I gave the user the option of naming their checkpoints. The names are stored in the list, alongside the copy of the Glitcher object. When the user reverts to a named checkpoint, the code searches through the list for that name, discards all checkpoints after the name, and sets that checkpoint image as the current image.

There is also a method for making a copy of a Glitcher object. As it stands, the copy does not have the checkpoints of the original, although I may implement that in the future.